In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [100]:
# read dataset into datasets
from datasets import Dataset
import pandas as pd
dataset_name = "five-star-trajectories"
dataset_path = Path(f"lm_act_eval/.cache/{dataset_name }")
traj_df = pd.read_csv(dataset_path/"csv/data.csv")
traj_dataset = Dataset.from_pandas(traj_df)
print(traj_df.columns)
traj_df[traj_df.session_id==traj_df.session_id.unique()[0]]
traj_df.head(2)

Index(['Unnamed: 0', '_id', 'tenant', 'project_id', 'created_at', 'updated_at',
       'inputs', 'history', 'ground_truth', 'linked_event', 'linked_evals',
       'linked_datasets', 'saved', 'type', 'metadata', 'QUERY', 'URL', 'DOM',
       'RULES', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'USER_CONTEXT', 'PREV_ACTIONS', 'prompt', 'CHAT_HISTORY'],
      dtype='object')


,Unnamed: 0,_id,tenant,project_id,created_at,updated_at,inputs,history,ground_truth,linked_event,...,DOM,RULES,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,USER_CONTEXT,PREV_ACTIONS,prompt,CHAT_HISTORY
0,0,65c95958a3d1e05a556776d4,org_5486oZLB8HZcT2Y1,648f6f2cd97aa222cef53da9,2024-02-11 23:33:44.479,2024-02-11 23:33:44.479,"{'QUERY': ""Book a reservation for the restaura...","[{'role': 'user', 'content': ""Book a reservati...",It appears that the browser content is not ava...,9c92b671-c113-4472-aeb9-d1d5944f1a9b,...,NaN,NaN,"Sunday, February 11, 2024 at 4:44:52 AM UTC","IMPORTANT: If the objective is complex, or you...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",NaN,NaN,NaN,NaN
1,1,65c95958a3d1e05a556776d5,org_5486oZLB8HZcT2Y1,648f6f2cd97aa222cef53da9,2024-02-11 23:33:44.480,2024-02-11 23:33:44.480,"{'QUERY': ""Book a reservation for the restaura...","[{'role': 'user', 'content': ""Book a reservati...",COMMANDS:\nCLICK 29\nEXPLANATION: I am clickin...,535be7d9-b6ad-4014-8191-a293ee71372d,...,<l id=0/>\n<t id=1>Skip to main content/>\n<l ...,NaN,"Sunday, February 11, 2024 at 4:44:48 AM UTC","IMPORTANT: If the objective is complex, or you...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",NaN,NaN,NaN,NaN


In [5]:
[
  print(obj[15]) for obj in [
    traj_df.outputs, 
    traj_df.ground_truth, 
    traj_df.PREV_ACTIONS
  ]
]

{'role': 'assistant', 'content': 'COMMANDS:\nCLICK 10\nEXPLANATION: I am clicking on the "Ayuda" (Help) section, which is likely to contain FAQs or other assistance information that may include details about the expiration of prepaid SIM cards.\n\nSTATUS: CONTINUE'}
COMMANDS:
CLICK 10
EXPLANATION: I am clicking on the "Ayuda" (Help) section, which is likely to contain FAQs or other assistance information that may include details about the expiration of prepaid SIM cards.

STATUS: CONTINUE
- I am navigating to the Claro Paraguay website to find information about the expiration of prepaid SIM cards.


- I am clicking on the "Ayuda" (Help) link, which is likely to lead to a section with customer support information or FAQs where we might find details about the expiration of prepaid SIM cards.




[None, None, None]

In [57]:
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible

In [15]:
print(extract_action(traj_df.ground_truth[211]))

GOTO_URL https://www.google.com/search?q=Tony+Kanaan+Indy+500+results+2013-2023


In [16]:
for i in [
  traj_df.PREV_ACTIONS,
  traj_df.previousActionsRepetitionPrompt,
  traj_df.chat_completion_messages,
  traj_df.ground_truth,
  traj_df.screenshot
  ]:
  if type(i[2])==str:
    print(i[2].replace('\n\n','\n'))
  else:
    print(i[2])
  print("")
  

- I am selecting Boogaloo - St. Louis restaurant which has been identified as accommodating vegetarian preferences to view their menu.


**PREVIOUS ACTIONS Repetition**:
- Review the "PREVIOUS ACTIONS"
- Then check if there is a repeated PREVIOUS ACTION.
- If you see repeated PREVIOUS ACTION, say: "EXPLANATION: I already did this action. I'm going to do another action.", and do another action.
- You can not repeat PREVIOUS ACTIONS, so you must correct yourself if you see a repeated PREVIOUS ACTION.
- Detect when you are in a loop based on the repeated PREVIOUS ACTIONS. You are a genius detecting that. This way you can win the game.

[{'role': 'system', 'content': '**Goal**: Let\'s play a game - You are an expert agent named MULTI·ON developed by "MultiOn" controlling a browser (you are not just a language model anymore). \n\nYou are given:\n1. An objective that you are trying to achieve\n2. The URL of your current web page\n3. A simplified text description of what\'s visible in the bro

In [4]:
from typing import Callable
from tqdm import tqdm

In [ ]:
# skip 
def criteria(eligibility_func: callable, cols: list, description: str=None):
  return pd.Series([eligibility_func(item) for item in tqdm(traj_df[cols], desc=description)])

def is_eligible():  
  # Apply the function to each URL in the column
  is_screenshot_accessible = criteria(is_screenshot_url_accessible, ['screenshot'], "determine URL accessibility")
  return is_screenshot_accessible

eligible = is_eligible()
# Calculate the percentage of accessible URLs
percentage_accessible = (eligible.sum() / len(traj_df)) * 100
print(percentage_accessible)


determine URL accessibility: 100%|██████████| 3382/3382 [1:02:27<00:00,  1.11s/it]

4.376108811354229


In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/"csv/data+gptv-eligible.csv", index_col=0)
# or 
# eligible_traj_df = traj_df[eligible]
len(eligible_traj_df)

148

In [90]:
from lm_act_eval.evaluation_harness.openai.vision.gptv import GPTV
from lm_act_eval.evaluation_harness.openai.vision.config import gptv_config
gptv_pipeline = GPTV(gptv_config)

In [31]:
groundtruth, chat_completions, screenshots, user_inputs = eligible_traj_df.ground_truth, eligible_traj_df.chat_completion_messages, eligible_traj_df.screenshot, eligible_traj_df.inputs
eligible_idx = eligible_traj_df.index

In [88]:
# Using as agent
from lm_act_eval.evaluation_harness.helper_functions.multion import extract_explanation

# Using as evaluator
eligible_traj_df['QUERY'] = user_inputs.apply(lambda s: ParseChatCompletion().parse_as_completion_content(s))

In [ ]:
eligible_traj_df = pd.read_csv(
  dataset_path/"csv/data+gptv-eligible.csv", index_col=0)
# or 
# eligible_traj_df = traj_df[eligible]
len(eligible_traj_df)

148

In [91]:
import warnings

completion_responses = []
for i in tqdm(eligible_idx, desc="Generating completions"):
  completion = gptv_pipeline(
    eligible_traj_df['QUERY'].loc[i],   #chat_completions_processed[i],
    screenshots.loc[i]
  )
  if len(completion) > 1:
    warnings.warn("Completion length more than 0 is unexpected")
  completion_responses.append(completion[0].message.content)
  

Generating completions:   0%|          | 0/148 [00:00<?, ?it/s]

Generating completions: 100%|██████████| 148/148 [21:43<00:00,  8.81s/it]


In [130]:
eligible_traj_df.loc[:, 'GPTV_generations'] = completion_responses
eligible_traj_df.GPTV_generations.iloc[0]

'PLAN: The current user objective is to find the actual enrollment count for a specific clinical trial regarding H. pylori and acne vulgaris patients from January to May 2018 as listed on the NIH website. Here are the steps to achieve this goal:\n\n1. Navigate to the NIH Clinical Trials website using the search query "site:clinicaltrials.gov H. pylori acne vulgaris".\n2. Use the website\'s search functionality to locate the specific trial conducted from Jan-May 2018.\n3. Locate the trial details page and find the actual enrollment count.\n4. If multiple search results appear, examine the relevant links to find the correct trial based on the date range specified in the user\'s request.\n5. If the information is not found after navigating the links, refine the search query or search through the trial histories.\n6. The enrollment information is typically available on the trial details page, usually under a section labeled "Participant Flow" or "Recruitment Status."\n\nI will start with t

## Save/Load Data

In [101]:
# traj_df_new = traj_df.merge(eligible_traj_df['GPTV-eval'], how='left')
# traj_df_new.to_csv(dataset_path/'csv/data+gptv.csv')

In [131]:
eligible_traj_df.to_csv(dataset_path/'csv/data+gptv-eligible.csv')

In [132]:
eligible_traj_df = pd.read_csv(dataset_path/'csv/data+gptv-eligible.csv', index_col=0)
eligible_traj_df.index.name=None
eligible_traj_df.head(2)

,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata,chat_completion_messages_content,GPTV_generations
169,36c2f0ba-e215-4081-96b4-ecb04c10a517,0,65ed4fc8166e07fe3c29c476,6c8932f3-c31d-4683-aeb0-52a5bbe5a978,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"Sunday, 10 March 2024 at 17:14:14 GMT+11","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN:\n1. Use the Google search function to fi...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,**Goal**: Let's play a game - You are an exper...,PLAN: The current user objective is to find th...
170,36c2f0ba-e215-4081-96b4-ecb04c10a517,1,65ed4fd1b2b5b0c1673c47d7,172ff25e-efdb-4002-b43a-0180f3f0eb19,36c2f0ba-e215-4081-96b4-ecb04c10a517,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'What was the actual enrollment coun...,...,"Sunday, 10 March 2024 at 17:14:35 GMT+11","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN,**Goal**: Let's play a game - You are an exper...,COMMANDS:\nCLICK 26\nEXPLANATION: I am clickin...


Logging

In [106]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [134]:
opentable_artifact = wandb.Artifact("opentable_trajectories_gptv", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(opentable_table_eligible, "opentable-eligibleonly")
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/'csv/data+gptv-eligible.csv'))


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

ArtifactManifestEntry(path='data+gptv-eligible.csv', digest='tbuWE+QMIHDMzBz9xKpwTw==', size=17643141, local_path='/home/opadmin/.local/share/wandb/artifacts/staging/tmpwfli3p9c')

In [135]:
# Start a W&B run to log data
run = wandb.init(
  project="opentable-vision", reinit=True)
# Log the table to visualize with a run...
run.log({
  "GPTV": opentable_table_eligible})

# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: WARNING Serializing object of type str that is 531752 bytes
wandb: WARNING Serializing object of type str that is 517908 bytes
wandb: WARNING Serializing object of type str that is 541284 bytes
wandb: WARNING Serializing object of type str that is 525810 bytes
wandb: WARNING Serializing object of type str that is 116864 bytes
wandb: WARNING Serializing object of type str that is 121274 bytes
wandb: WARNING Serializing object of type str that is 120688 bytes
wandb: WARNING Serializing object of type str that is 124364 bytes
wandb: WARNING Serializing object of type str that is 127354 bytes
wandb: WARNING Serializing object of type str that is 129308 bytes
wandb: WARNING Serializing object of type str that is 129582 bytes
wandb: WARNING Serializing object of type str that is 135310 bytes
wandb: WARNING Serializing object of type str that is 137856 bytes
wandb: WARNING Serializing object of type str that is 137644 bytes
wandb: WARNING Serializing object of type str that is 136800 b

<Artifact opentable_trajectories_gptv>